In [1]:
import sys
sys.path.append("/home/tmartorella/ddpm")

from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
from torchmetrics.image import StructuralSimilarityIndexMeasure
# from basicsr.metrics.metric_util import reorder_image, to_y_channel
from ddpm.datasets.celebahq import CelebAHQ
from torchvision import transforms
from PIL import Image
import numpy as np
import cv2

t = transforms.Compose([transforms.ToTensor(), transforms.Resize([256]), transforms.CenterCrop([256, 256])])
# dataset = CelebAHQ(root="/home/bastien/scitas/bastien/CelebAMask-HQ/CelebA-HQ-img/", transform=t)

def to_pil(img):
    return Image.fromarray((img * 255).permute(1, 2, 0).numpy().astype(np.uint8))


# def _ssim(img1, img2):
#     """Calculate SSIM (structural similarity) for one channel images.

#     It is called by func:`calculate_ssim`.

#     Args:
#         img1 (ndarray): Images with range [0, 255] with order 'HWC'.
#         img2 (ndarray): Images with range [0, 255] with order 'HWC'.

#     Returns:
#         float: ssim result.
#     """

#     C1 = (0.01 * 255)**2
#     C2 = (0.03 * 255)**2

#     img1 = img1.astype(np.float64)
#     img2 = img2.astype(np.float64)
#     kernel = cv2.getGaussianKernel(11, 1.5)
#     window = np.outer(kernel, kernel.transpose())

#     mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]
#     mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
#     mu1_sq = mu1**2
#     mu2_sq = mu2**2
#     mu1_mu2 = mu1 * mu2
#     sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
#     sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
#     sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

#     ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
#     return ssim_map.mean()


# def calculate_ssim(img1, img2, crop_border, input_order='HWC', test_y_channel=False):
#     """Calculate SSIM (structural similarity).

#     Ref:
#     Image quality assessment: From error visibility to structural similarity

#     The results are the same as that of the official released MATLAB code in
#     https://ece.uwaterloo.ca/~z70wang/research/ssim/.

#     For three-channel images, SSIM is calculated for each channel and then
#     averaged.

#     Args:
#         img1 (ndarray): Images with range [0, 255].
#         img2 (ndarray): Images with range [0, 255].
#         crop_border (int): Cropped pixels in each edge of an image. These
#             pixels are not involved in the SSIM calculation.
#         input_order (str): Whether the input order is 'HWC' or 'CHW'.
#             Default: 'HWC'.
#         test_y_channel (bool): Test on Y channel of YCbCr. Default: False.

#     Returns:
#         float: ssim result.
#     """

#     assert img1.shape == img2.shape, (f'Image shapes are differnet: {img1.shape}, {img2.shape}.')
#     if input_order not in ['HWC', 'CHW']:
#         raise ValueError(f'Wrong input_order {input_order}. Supported input_orders are ' '"HWC" and "CHW"')
#     img1 = reorder_image(img1, input_order=input_order)
#     img2 = reorder_image(img2, input_order=input_order)
#     img1 = img1.astype(np.float64)
#     img2 = img2.astype(np.float64)

#     if crop_border != 0:
#         img1 = img1[crop_border:-crop_border, crop_border:-crop_border, ...]
#         img2 = img2[crop_border:-crop_border, crop_border:-crop_border, ...]

#     if test_y_channel:
#         img1 = to_y_channel(img1)
#         img2 = to_y_channel(img2)

#     ssims = []
#     for i in range(img1.shape[2]):
#         ssims.append(_ssim(img1[..., i], img2[..., i]))
#     return np.array(ssims).mean()

# def calculate_ssim(img1, img2, crop_border):
#     ssim = StructuralSimilarityIndexMeasure(data_range=255)
#     ssim(img1, img2)


# def get_metrics(img1, img2):
#     ssim = calculate_ssim(img1, img2, 0)
#     psnr = calculate_psnr(img1, img2, 0)
#     return ssim, psnr

In [175]:
from tqdm import tqdm
import torch
from torchmetrics.image import (
    PeakSignalNoiseRatio,
    StructuralSimilarityIndexMeasure,
)

from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity

def get_filtering_metrics(df):
    metrics = []
    
    psnr_tgt_metric = PeakSignalNoiseRatio().to("cuda")
    lpips_tgt_metric = LearnedPerceptualImagePatchSimilarity().to("cuda")

    for i, img in tqdm(df.iterrows(), total=len(df)):
        try:
            # src = t(Image.open(img["path"].parents[0 if img["algo"] == "none" else 2] / "corrupted.png")).to("cuda")
            tgt = t(Image.open(Path(img["path"]).parents[0 if img["algo"] == "none" else 2] / "original.png")).to("cuda")
            img = t(Image.open(img["path"])).to("cuda")

            
            # psnr_tgt_metric.update(img, tgt)
            # lpips_tgt_metric.update((img * 2 - 1).unsqueeze(0), (tgt * 2 - 1).unsqueeze(0))

            metrics.append({
                "psnr_to_target": psnr_tgt_metric(img, tgt).item(),
                "lpips_to_target": lpips_tgt_metric((img * 2 - 1).unsqueeze(0), (tgt * 2 - 1).unsqueeze(0)).item(),
            })  
        except Exception as e:
            print(f"error with image {i} - {e}")
    
    return pd.DataFrame(metrics)

def get_metrics(df, fid_metric, other_groups=[]):
    metrics = []
    
    for g in tqdm(df.groupby(["corruption"]+other_groups)):
        c = g[0]    
        
        psnr_src_metric = PeakSignalNoiseRatio().to("cuda")
        psnr_tgt_metric = PeakSignalNoiseRatio().to("cuda")
        ssim_src_metric = StructuralSimilarityIndexMeasure(data_range=(0, 1)).to("cuda")
        ssim_tgt_metric = StructuralSimilarityIndexMeasure(data_range=(0, 1)).to("cuda")
        lpips_src_metric = LearnedPerceptualImagePatchSimilarity().to("cuda")
        lpips_tgt_metric = LearnedPerceptualImagePatchSimilarity().to("cuda")
        l2_src_metric = []
        l2_tgt_metric = []
        fid_metric.reset()
        
        list_imgs = list(g[1].iterrows())
        for i, img in list_imgs:
            try:
                src = t(Image.open(Path(img["path"]).parents[0 if img["algo"] == "none" else 2] / "corrupted.png")).to("cuda")
                tgt = t(Image.open(Path(img["path"]).parents[0 if img["algo"] == "none" else 2] / "original.png")).to("cuda")
                img = t(Image.open(img["path"])).to("cuda")
                
                psnr_src_metric.update(img, src)
                psnr_tgt_metric.update(img, tgt)
                ssim_src_metric.update(img.unsqueeze(0), src.unsqueeze(0))
                ssim_tgt_metric.update(img.unsqueeze(0), tgt.unsqueeze(0))
                lpips_src_metric.update((img * 2 - 1).unsqueeze(0), (src * 2 - 1).unsqueeze(0))
                lpips_tgt_metric.update((img * 2 - 1).unsqueeze(0), (tgt * 2 - 1).unsqueeze(0))
                l2_src_metric.append(torch.linalg.norm((img - src).reshape(-1)))
                l2_tgt_metric.append(torch.linalg.norm((img - tgt).reshape(-1)))
                fid_metric.update((img * 255).to(torch.uint8).unsqueeze(0), real=False)
            except Exception as e:
                print(f"error with image {i} - {e}")
    
        psnr_to_source = psnr_src_metric.compute()
        psnr_to_target = psnr_tgt_metric.compute()
        ssim_to_source = ssim_src_metric.compute()
        ssim_to_target = ssim_tgt_metric.compute()
        l2_to_source = torch.stack(l2_src_metric).mean()
        l2_to_target = torch.stack(l2_tgt_metric).mean()
        lpips_to_source = lpips_src_metric.compute()
        lpips_to_target = lpips_tgt_metric.compute()
        fid = fid_metric.compute()
        
        metrics.append({
            "corruption": c[0],
            "psnr_to_input": psnr_to_source.item(),
            "psnr_to_target": psnr_to_target.item(),
            "ssim_to_input": ssim_to_source.item(),
            "ssim_to_target": ssim_to_target.item(),
            "l2_to_input": l2_to_source.item(),
            "l2_to_target": l2_to_target.item(),
            "lpips_to_input": lpips_to_source.item(),
            "lpips_to_target": lpips_to_target.item(),
            "fid": fid.item(),
            "num_imgs": len(list(g[1].iterrows()))
        })
        
    return pd.DataFrame(metrics)

In [166]:
from pathlib import Path
from tqdm.auto import tqdm
import os
import fnmatch

files = []
path_list = [
    # "/home/bastien/scitas_2/bastien/results_CODE/results/ODEDIT/CELEBAHQ/fid/good_ones",
    "/home/bastien/rcp/scratch/vita-staff/users/bvandelft/results/ODEDIT/CELEBAHQ/Celeba",
    "/home/bastien/rcp/scratch/vita-staff/users/bvandelft/results/ODEDIT/CELEBAHQ/Celeba2"
]

# for p in path_list:
#     files += [f for f in tqdm(list(Path(p).glob("**/*.png"))) if f.is_file()]
# files[:20]
files = []
for p in path_list:
    for root, dirnames, filenames in tqdm(os.walk(p)):
        for filename in fnmatch.filter(filenames, '*.png'):
            files.append(Path(os.path.join(root, filename)))
    #         if len(files) > 2000:
    #             break
    #     if len(files) > 2000:
    #         break
    # if len(files) > 2000:
    #     break

print(files[:20])

# files_scitas = []
# path_list = [
#     "/home/bastien/scitas/bastien/results_CODE/results/ODEDIT/CELEBAHQ/fid/good_ones",
#     # "/home/bastien/rcp/scratch/vita-staff/users/bvandelft/results/ODEDIT/CELEBAHQ/Celeba"
# ]

# for p in path_list:
#     files_scitas += [f for f in tqdm(list(Path(p).glob("**/*.png"))) if f.is_file()]
# files[:20]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[PosixPath('/home/bastien/rcp/scratch/vita-staff/users/bvandelft/results/ODEDIT/CELEBAHQ/Celeba/2024-07-04_11-45/pixelate/15053/corrupted.png'), PosixPath('/home/bastien/rcp/scratch/vita-staff/users/bvandelft/results/ODEDIT/CELEBAHQ/Celeba/2024-07-04_11-45/pixelate/15053/original.png'), PosixPath('/home/bastien/rcp/scratch/vita-staff/users/bvandelft/results/ODEDIT/CELEBAHQ/Celeba/2024-07-04_11-45/pixelate/15053/ode/clipped/200_0.002_0.png'), PosixPath('/home/bastien/rcp/scratch/vita-staff/users/bvandelft/results/ODEDIT/CELEBAHQ/Celeba/2024-07-04_11-45/pixelate/15053/ode/non_clipped/200_0.0001_0.png'), PosixPath('/home/bastien/rcp/scratch/vita-staff/users/bvandelft/results/ODEDIT/CELEBAHQ/Celeba/2024-07-04_11-45/pixelate/15053/ode/non_clipped/200_0.000271442_0.png'), PosixPath('/home/bastien/rcp/scratch/vita-staff/users/bvandelft/results/ODEDIT/CELEBAHQ/Celeba/2024-07-04_11-45/pixelate/15053/ode/non_clipped/200_0.000736806_0.png'), PosixPath('/home/bastien/rcp/scratch/vita-staff/users/b

In [167]:
import pandas as pd
import numpy as np

info = []
for f in files:
    f_split = str(f).split("/")
    if f.name in ["corrupted.png", "original.png"]:
        info.append({
            "path": f,
            "corruption": f_split[-3],
            "img_id": f_split[-2],
            "filename": f_split[-1],
            "algo": "none",
            "clipping": "none",
            "timestamp": f_split[-4],
            "latent": "none",
            "epsilon": np.nan,
        })
    else:
        info.append({
            "path": f,
            "corruption": f_split[-5],
            "img_id": f_split[-4],
            "filename": f_split[-1],
            "algo": f_split[-3],
            "clipping": f_split[-2],
            "timestamp": f_split[-6],
            "latent": f_split[-1].split("_")[0] if f_split[-3] == "sde" or f_split[-3] == "ode" else "none",
            "epsilon": f_split[-1].split("_")[1] if f_split[-3] == "ode" else np.nan
        })

df = pd.DataFrame(info)
df['epsilon'] = pd.cut(
    df['epsilon'].astype(np.float64),
    bins=[1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1],
    labels=[1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2],
    right=False
)

In [168]:
df

,path,corruption,img_id,filename,algo,clipping,timestamp,latent,epsilon
0,/home/bastien/rcp/scratch/vita-staff/users/bva...,pixelate,15053,corrupted.png,none,none,2024-07-04_11-45,none,NaN
1,/home/bastien/rcp/scratch/vita-staff/users/bva...,pixelate,15053,original.png,none,none,2024-07-04_11-45,none,NaN
2,/home/bastien/rcp/scratch/vita-staff/users/bva...,pixelate,15053,200_0.002_0.png,ode,clipped,2024-07-04_11-45,200,0.00100
3,/home/bastien/rcp/scratch/vita-staff/users/bva...,pixelate,15053,200_0.0001_0.png,ode,non_clipped,2024-07-04_11-45,200,0.00010
4,/home/bastien/rcp/scratch/vita-staff/users/bva...,pixelate,15053,200_0.000271442_0.png,ode,non_clipped,2024-07-04_11-45,200,0.00010
...,...,...,...,...,...,...,...,...,...
1115176,/home/bastien/rcp/scratch/vita-staff/users/bva...,technicolor,6811,200_7.9527e-05_1.png,ode,non_clipped,2024-08-03_14-00,200,0.00005
1115177,/home/bastien/rcp/scratch/vita-staff/users/bva...,technicolor,6811,200_0.0002_0.png,ode,non_clipped,2024-08-03_14-00,200,0.00010
1115178,/home/bastien/rcp/scratch/vita-staff/users/bva...,technicolor,6811,200_0.0002_1.png,ode,non_clipped,2024-08-03_14-00,200,0.00010
1115179,/home/bastien/rcp/scratch/vita-staff/users/bva...,technicolor,6811,reconstruction_ddim.png,reconstruction,clipped,2024-08-03_14-00,none,NaN


In [169]:
df[(df['algo'] == "ode") & (df['clipping'] == 'non_clipped')].groupby("corruption")['img_id'].unique().apply(len)

corruption
bleach_bypass                       572
blue_noise                          543
blue_noise_sample                   572
brightness                          572
brownish_noise                      572
caustic_noise                       572
caustic_refraction                  543
checkerboard_cutout                1056
chromatic_abberation                572
circular_motion_blur                543
cocentric_sine_waves                572
color_dither                        572
contrast                            572
elastic_transform                   870
fish_eye                            324
fog                                 543
frost                               870
gaussian_blur                       870
gaussian_noise                      870
glass_blur                          572
hue_shift                           324
impulse_noise                       572
inverse_sparkles                    870
jpeg_compression                    870
lines                        

In [170]:
df[(df['algo'] == "ode") & (df['clipping'] == 'clipped')].groupby("corruption")['img_id'].unique().apply(len)

corruption
bleach_bypass                       552
blue_noise                          542
blue_noise_sample                   552
brightness                          552
brownish_noise                      552
caustic_noise                       552
caustic_refraction                  542
checkerboard_cutout                1051
chromatic_abberation                552
circular_motion_blur                542
cocentric_sine_waves                552
color_dither                        552
contrast                            552
elastic_transform                   857
fish_eye                            313
fog                                 542
frost                               857
gaussian_blur                       857
gaussian_noise                      857
glass_blur                          552
hue_shift                           313
impulse_noise                       552
inverse_sparkles                    857
jpeg_compression                    857
lines                        

In [171]:
df[(df['algo'] == "sde") & (df['latent'] == "699")].groupby("corruption")['img_id'].unique().apply(len)

corruption
bleach_bypass                       584
blue_noise                          543
blue_noise_sample                   584
brightness                          584
brownish_noise                      584
caustic_noise                       584
caustic_refraction                  543
checkerboard_cutout                1055
chromatic_abberation                584
circular_motion_blur                543
cocentric_sine_waves                584
color_dither                        584
contrast                            584
elastic_transform                   875
fish_eye                            330
fog                                 543
frost                               875
gaussian_blur                       875
gaussian_noise                      875
glass_blur                          584
hue_shift                           330
impulse_noise                       584
inverse_sparkles                    875
jpeg_compression                    875
lines                        

In [172]:
df[(df['algo'] == "sde") & (df['latent'] == "299")].groupby("corruption")['img_id'].unique().apply(len)

corruption
bleach_bypass                      611
blue_noise                         544
blue_noise_sample                  611
brightness                         611
brownish_noise                     611
caustic_noise                      611
caustic_refraction                 544
checkerboard_cutout                544
chromatic_abberation               611
circular_motion_blur               544
cocentric_sine_waves               611
color_dither                       611
contrast                           611
elastic_transform                  378
fish_eye                           356
fog                                544
frost                              378
gaussian_blur                      378
gaussian_noise                     378
glass_blur                         613
hue_shift                          356
impulse_noise                      612
inverse_sparkles                   378
jpeg_compression                   378
lines                              356
masking_random

In [173]:
df['corruption'].unique()

array(['pixelate', 'gaussian_blur', 'elastic_transform', 'shot_noise',
       'gaussian_noise', 'frost', 'jpeg_compression',
       'masking_random_color', 'perlin_noise', 'checkerboard_cutout',
       'plasma_noise', 'transverse_chromatic_abberation',
       'inverse_sparkles', 'fog', 'masking_vline_random_color',
       'blue_noise', 'caustic_refraction', 'ripple',
       'circular_motion_blur', 'water_drop', 'voronoi_noise',
       'pinch_and_twirl', 'glass_blur', 'impulse_noise', 'contrast',
       'brightness', 'saturate', 'motion_blur', 'speckle_noise',
       'bleach_bypass', 'blue_noise_sample', 'brownish_noise',
       'caustic_noise', 'chromatic_abberation', 'color_dither',
       'cocentric_sine_waves', 'snow', 'spatter', 'fish_eye', 'hue_shift',
       'lines', 'perspective_no_bars', 'quadrilateral_no_bars',
       'pseudocolor', 'scatter', 'single_frequency_greyscale',
       'technicolor'], dtype=object)

In [ ]:
df_with_metrics = pd.concat([df, get_filtering_metrics(df)], axis=1)

 12%|████████                                                             | 130724/1115181 [1:06:16<10:34:04, 25.88it/s]

In [ ]:
df_with_metrics.to_csv("./df_with_metrics.csv")
df_with_metrics

# top 1 and top 4 by psnr

In [ ]:
######################
# now top 1

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping'])
    .apply(lambda x: x.sort_values('psnr_to_target', ascending=False).head(1), include_groups=False)
    .reset_index().drop(columns="level_4")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_1_imgs_by_psnr.csv")
display(df_with_metrics_filtered.head(5))


#########################
# now top 4

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping'])
    .apply(lambda x: x.sort_values('psnr_to_target', ascending=False).head(4), include_groups=False)
    .reset_index().drop(columns="level_4")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_4_imgs_by_psnr.csv")
display(df_with_metrics_filtered.head(5))

# top 1 and top 4 by lpips

In [ ]:
######################
# now top 1

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping'])
    .apply(lambda x: x.sort_values('lpips_to_target', ascending=True).head(1), include_groups=False)
    .reset_index().drop(columns="level_4")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_1_imgs_by_lpips.csv")
display(df_with_metrics_filtered.head(5))


#########################
# now top 4

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping'])
    .apply(lambda x: x.sort_values('lpips_to_target', ascending=True).head(4), include_groups=False)
    .reset_index().drop(columns="level_4")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_4_imgs_by_lpips.csv")
display(df_with_metrics_filtered.head(5))

# top 1 and top 4 by psnr per latent and epsilon

In [ ]:
######################
# now top 1

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping', 'latent', 'epsilon'], observed=False)
    .apply(lambda x: x.sort_values('psnr_to_target', ascending=False).head(1), include_groups=False)
    .reset_index().drop(columns="level_6")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_1_imgs_by_psnr_by_latent_and_epsilon.csv")
display(df_with_metrics_filtered.head(5))


#########################
# now top 4

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping', 'latent', 'epsilon'], observed=False)
    .apply(lambda x: x.sort_values('psnr_to_target', ascending=False).head(4), include_groups=False)
    .reset_index().drop(columns="level_6")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_4_imgs_by_psnr_by_latent_and_epsilon.csv")
display(df_with_metrics_filtered.head(5))

# top 1 and top 4 by lpips by latent and epsilon

In [ ]:
######################
# now top 1

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping', 'latent', 'epsilon'], observed=False)
    .apply(lambda x: x.sort_values('lpips_to_target', ascending=True).head(1), include_groups=False)
    .reset_index().drop(columns="level_6")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_1_imgs_by_lpips_by_latent_and_epsilon.csv")
display(df_with_metrics_filtered.head(5))


#########################
# now top 4

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping', 'latent', 'epsilon'], observed=False)
    .apply(lambda x: x.sort_values('lpips_to_target', ascending=True).head(4), include_groups=False)
    .reset_index().drop(columns="level_6")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_4_imgs_by_lpips_by_latent_and_epsilon.csv")
display(df_with_metrics_filtered.head(5))

# Calculate Metrics

In [ ]:
import pandas as pd

df = pd.read_csv('./top_4_imgs_by_psnr_by_latent_and_epsilon.csv')

In [ ]:
dfs = {
    "reconstruction_without_clipping": {
        "df": df[(df["algo"] == "reconstruction") & (df["clipping"] == "non_clipped")].copy().drop(columns=["timestamp"]).copy()
    },
    "reconstruction_with_clipping": {
        "df": df[(df["algo"] == "reconstruction") & (df["clipping"] == "clipped")].copy().drop(columns=["timestamp"]).copy()
    },
    "ode_without_clipping": {
        "df": df[(df["algo"] == "ode") & (df["clipping"] == "non_clipped")].copy().drop(columns=["timestamp"]).copy()
    },
    "ode_with_clipping": {
        "df": df[(df["algo"] == "ode") & (df["clipping"] == "clipped")].copy().drop(columns=["timestamp"]).copy()
    },
    "sde_without_clipping": {
        "df": df[(df["algo"] == "sde") & (df["clipping"] == "non_clipped")].copy().drop(columns=["timestamp"]).copy()
    },
    "original": {
        "df": df[(df["algo"] == "none") & (df["filename"] == "original.png")].copy().drop(columns=["timestamp"]).copy()
    },
    "corrupted": {
        "df": df[(df["algo"] == "none") & (df["filename"] == "corrupted.png")].copy().drop(columns=["timestamp"]).copy()
    },
}

In [112]:

import sys
sys.path.append("/home/tmartorella/ddpm")

from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
from torchmetrics.image import StructuralSimilarityIndexMeasure
# from basicsr.metrics.metric_util import reorder_image, to_y_channel
from ddpm.datasets.celebahq import CelebAHQ
from torchvision import transforms
from PIL import Image
import numpy as np
import cv2

celebahq_fid = FrechetInceptionDistance(reset_real_features=False).to("cuda")

import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from ddpm.datasets.celebahq import CelebAHQ
from torchvision import transforms
t =  transforms.Compose([transforms.ToTensor(), transforms.Resize([256]), transforms.CenterCrop([256, 256])])

class ADataset(Dataset):
    def __init__(self, ds):
        self.ds = ds
        
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, idx):
        return self.ds[idx][0]

celeba_dataset = CelebAHQ(root="/home/bastien/rcp/scratch/datasets/CelebAMask-HQ/CelebA-HQ-img/", transform=t)
loader = DataLoader(ADataset(celeba_dataset), batch_size=16, num_workers=4, pin_memory=True)

for batch in tqdm(loader):
    celebahq_fid.update((batch * 255).to(torch.uint8).to("cuda"), real=True)

100%|███████████████████████████████████████████████████████████████████████████████| 1875/1875 [07:42<00:00,  4.05it/s]


In [ ]:
from IPython.display import display, Markdown

for k, v in dfs.items():
    print(k)
    
    metrics_per_corruption = get_metrics(v["df"].copy(), celebahq_fid)
    display(Markdown(f"## {k}"))
    display(metrics_per_corruption)
    metrics_per_corruption.to_csv(f"./metrics_per_corruption/{k}_per_corruption.csv")

    v["metrics_per_corruption"] = metrics_per_corruption

    _fake_df = v["df"].copy()

    metrics_aggregated = get_metrics(_fake_df.copy(), celebahq_fid, ['epsilon', 'latent'])
    display(Markdown(f"## {k} - aggregated"))
    display(metrics_aggregated)
    metrics_per_corruption.to_csv(f"./metrics_per_epsilon-latent/{k}_aggregated.csv")
    
    v["metrics_per_corruption_and_epsilon_and_latent"] = metrics_aggregated

    _fake_df = v["df"].copy()
    _fake_df["corruption"] = "all"

    metrics_aggregated = get_metrics(_fake_df.copy(), celebahq_fid)
    display(Markdown(f"## {k} - aggregated"))
    display(metrics_aggregated)
    metrics_per_corruption.to_csv(f"./metrics_aggregated/{k}_aggregated.csv")
    
    v["metrics_aggregated"] = metrics_aggregated

# Fin fin

In [12]:
from IPython.display import display, Markdown

for k, v in dfs.items():
    print(k)
    if not k.startswith("ode"):
        continue
    
    metrics_per_corruption = get_metrics(v["df"].copy(), celebahq_fid)
    display(Markdown(f"## {k}"))
    display(metrics_per_corruption)

    v["metrics_per_corruption"] = metrics_per_corruption

    _fake_df = v["df"].copy()
    _fake_df["corruption"] = "all"

    metrics_aggregated = get_metrics(_fake_df.copy(), celebahq_fid)
    display(Markdown(f"## {k} - aggregated"))
    display(metrics_aggregated)
    
    v["metrics_aggregated"] = metrics_aggregated

reconstruction_without_clipping
reconstruction_with_clipping
ode_without_clipping


100%|████████████████████████████████████████████████████████████████████████████████| 22/22 [1:03:08<00:00, 172.20s/it]


## ode_without_clipping

,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid,num_imgs
0,blue_noise,10.668242,18.899797,0.046660,0.484915,129.681503,48.740536,1.191962,0.295653,26.832867,2672
1,caustic_refraction,16.607018,16.467705,0.420130,0.437634,61.969551,63.143482,0.370441,0.365928,35.293240,2672
2,checkerboard_cutout,16.364853,14.571896,0.483814,0.445621,63.652260,80.543564,0.466347,0.406823,28.776552,7873
3,circular_motion_blur,17.647478,16.327309,0.579409,0.467320,52.814022,63.786526,0.375388,0.355428,30.079262,2672
4,elastic_transform,17.374796,16.365879,0.517543,0.455989,56.076702,64.701851,0.363616,0.342433,24.196011,5201
5,fog,17.413744,10.924592,0.542183,0.321835,56.205074,123.915649,0.528568,0.581771,52.543270,2672
6,frost,19.369930,8.722367,0.415221,0.381307,44.189400,157.004868,0.522870,0.522388,42.089340,5201
7,gaussian_blur,17.613876,15.924047,0.643830,0.449541,53.506531,67.544685,0.569331,0.384652,21.204124,5201
8,gaussian_noise,10.975690,19.489967,0.109997,0.475424,124.876328,45.961975,1.193336,0.358731,25.500141,5201
9,inverse_sparkles,16.835379,8.858855,0.474134,0.367237,60.865173,156.310486,0.465972,0.506856,62.140598,5201


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [38:47<00:00, 2327.32s/it]


...

,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid,num_imgs
0,blue_noise,10.370693,18.863174,0.030520,0.523801,134.198303,46.230663,1.238844,0.268156,25.497595,2662
1,caustic_refraction,17.615057,17.737501,0.439630,0.477476,55.655613,54.776535,0.356488,0.330565,26.028574,2662
2,checkerboard_cutout,17.171724,15.796113,0.502950,0.484163,58.991917,70.074348,0.467108,0.358999,19.028614,7810
3,circular_motion_blur,19.356211,17.713488,0.610239,0.494226,43.758125,54.905373,0.348701,0.321844,28.938641,2662
4,elastic_transform,18.413212,17.502090,0.548152,0.509746,50.257801,56.906914,0.352651,0.300993,19.954062,5148
5,fog,18.354589,11.085973,0.567645,0.327373,50.625675,121.799774,0.514666,0.588637,53.974277,2660
6,frost,19.426559,8.591591,0.404543,0.396494,44.709225,158.447784,0.545519,0.521340,39.295033,5148
7,gaussian_blur,19.399809,17.188837,0.678240,0.477336,44.089603,58.998108,0.542359,0.351771,24.267345,5148
8,gaussian_noise,10.705048,20.540096,0.063298,0.529900,129.104889,40.457214,1.280733,0.265864,19.486181,5148
9,inverse_sparkles,17.818531,8.949427,0.483183,0.391937,54.909897,155.156372,0.483104,0.507157,55.638527,5148


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [39:11<00:00, 2351.81s/it]


## ode_with_clipping - aggregated

,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid,num_imgs
0,all,14.850633,14.248871,0.409495,0.480319,71.34568,75.221275,0.641683,0.367298,15.668028,96204


sde_without_clipping
original
corrupted


In [13]:
import pickle
with open("/home/bastien/Projects/ddpm/metrics/metrics_new_final.pkl", "wb") as metrics_file:
    pickle.dump(dfs, metrics_file)

In [ ]:
from IPython.display import display, Markdown

for k, v in dfs.items():
    print(k)
    if not k.startswith("ode"):
        continue
    
    metrics_per_corruption = get_metrics(v["df"].copy(), celebahq_fid)
    display(Markdown(f"## {k}"))
    display(metrics_per_corruption)

    v["metrics_per_corruption"] = metrics_per_corruption

    _fake_df = v["df"].copy()
    _fake_df["corruption"] = "all"

    metrics_aggregated = get_metrics(_fake_df.copy(), celebahq_fid)
    display(Markdown(f"## {k} - aggregated"))
    display(metrics_aggregated)
    
    v["metrics_aggregated"] = metrics_aggregated

reconstruction_without_clipping
reconstruction_with_clipping
ode_without_clipping


 38%|██████████████████████████████                                                  | 15/40 [40:50<1:05:26, 157.05s/it]

error with image 42783
error with image 96805


 42%|██████████████████████████████████                                              | 17/40 [51:47<1:29:52, 234.44s/it]

error with image 84551
error with image 84552
error with image 84553


...

ode_with_clipping


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [57:58<00:00, 158.11s/it]


## ode_with_clipping

,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid,num_imgs
0,blue_noise,10.373595,18.872232,0.030679,0.524792,134.154343,46.166546,1.237647,0.266157,25.961205,2474
1,caustic_refraction,17.621508,17.744844,0.440761,0.478770,55.597073,54.707382,0.354590,0.327686,26.589403,2474
2,checkerboard_cutout,17.167625,15.799060,0.504024,0.485494,59.020737,70.038872,0.467003,0.358383,19.114693,7622
3,circular_motion_blur,19.351984,17.718204,0.610494,0.495221,43.776684,54.855000,0.347654,0.318667,29.337038,2474
4,elastic_transform,18.413212,17.502090,0.548152,0.509746,50.257801,56.906914,0.352651,0.300993,19.954062,5148
5,fog,18.334143,11.098530,0.567491,0.328090,50.762455,121.576157,0.513970,0.585761,54.654873,2474
6,frost,19.426559,8.591591,0.404543,0.396494,44.709225,158.447784,0.545519,0.521340,39.295033,5148
7,gaussian_blur,19.399809,17.188837,0.678240,0.477336,44.089603,58.998108,0.542359,0.351771,24.267345,5148
8,gaussian_noise,10.705048,20.540096,0.063298,0.529900,129.104889,40.457214,1.280733,0.265864,19.486181,5148
9,inverse_sparkles,17.818531,8.949427,0.483183,0.391937,54.909897,155.156372,0.483104,0.507157,55.638527,5148


  0%|                                                                                             | 0/1 [00:00<?, ?it/s]

In [ ]:
import pickle
with open("/home/bastien/Projects/ddpm/metrics/metrics_new_final.pkl", "wb") as metrics_file:
    pickle.dump(dfs, metrics_file)

## Fin

In [29]:
woof = ("/home/bastien/Projects/ddpm/metrics/metrics_new.pkl")

In [36]:
woof['original']['metrics_aggregated']

,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid,num_imgs
0,all,14.505411,inf,0.549664,1.0,71.940269,0.0,0.468557,0.0,18.665163,18075


In [91]:
from IPython.display import display, Markdown

for k, v in dfs.items():
    # if k not in ["corrupted", "original"]:
    #     continue
    
    metrics_per_corruption = get_metrics(v["df"].copy(), celebahq_fid)
    display(Markdown(f"## {k}"))
    display(metrics_per_corruption)

    v["metrics_per_corruption"] = metrics_per_corruption

    _fake_df = v["df"].copy()
    _fake_df["corruption"] = "all"

    metrics_aggregated = get_metrics(_fake_df.copy(), celebahq_fid)
    display(Markdown(f"## {k} - aggregated"))
    display(metrics_aggregated)
    
    v["metrics_aggregated"] = metrics_aggregated

  0%|          | 0/16 [00:00<?, ?it/s]

## original

,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid,num_imgs
0,brownish_noise,16.859682,inf,0.795461,1.000000,63.374878,0.0,0.283796,0.0,136.829468,16
1,caustic_noise,18.234344,inf,0.849742,1.000000,54.236118,0.0,0.099865,0.0,136.829468,16
2,caustic_refraction,21.453802,inf,0.757663,1.000000,37.165993,0.0,0.192048,0.0,136.829468,16
3,checkerboard_cutout,16.495350,inf,0.780126,0.999997,64.899422,0.0,0.341479,0.0,50.443966,206
4,cocentric_sine_waves,23.022499,inf,0.578347,1.000000,31.305069,0.0,0.349037,0.0,136.829468,16
5,fish_eye,18.183718,inf,0.470227,1.000000,54.175911,0.0,0.464492,0.0,136.829468,16
6,gaussian_blur,22.162159,inf,0.587452,1.000000,34.142033,0.0,0.630953,0.0,136.829468,16
7,inverse_sparkles,9.964433,inf,0.578729,1.000000,138.519394,0.0,0.433153,0.0,136.829468,16
8,masking_random_color,11.802941,inf,0.243072,0.999997,111.564552,0.0,1.071532,0.0,50.443966,206
9,masking_vline_random_color,11.639180,inf,0.165897,0.999997,115.654999,0.0,0.870197,0.0,50.443966,206


  0%|          | 0/1 [00:00<?, ?it/s]

## original - aggregated

...

## corrupted

,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid,num_imgs
0,brownish_noise,inf,16.859682,1.000000,0.795461,0.0,63.374878,0.0,0.283796,169.313644,16
1,caustic_noise,inf,18.234344,1.000000,0.849742,0.0,54.236118,0.0,0.099865,154.926849,16
2,caustic_refraction,inf,21.453802,1.000000,0.757663,0.0,37.165993,0.0,0.192048,167.762604,16
3,checkerboard_cutout,inf,16.495350,0.999998,0.780126,0.0,64.899422,0.0,0.341479,270.342499,206
4,cocentric_sine_waves,inf,23.022499,1.000000,0.578347,0.0,31.305069,0.0,0.349037,337.507294,16
5,fish_eye,inf,18.183718,0.999992,0.470227,0.0,54.175911,0.0,0.464492,408.461365,16
6,gaussian_blur,inf,22.196289,0.999995,0.587452,0.0,34.142033,0.0,0.630953,231.149490,16
7,inverse_sparkles,inf,9.964433,0.999999,0.578729,0.0,138.519394,0.0,0.433153,303.855286,16
8,masking_random_color,inf,11.802941,1.000000,0.243072,0.0,111.564552,0.0,1.071532,230.866531,206
9,masking_vline_random_color,inf,11.639180,1.000000,0.165897,0.0,115.654999,0.0,0.870197,355.248138,206


  0%|          | 0/1 [00:00<?, ?it/s]

## corrupted - aggregated

,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid,num_imgs
0,all,inf,14.746831,1.0,0.563867,0.0,73.49295,0.0,0.502755,127.835823,1206


In [93]:
from IPython.display import display, Markdown

for k, v in dfs.items():
    # if k not in ["corrupted", "original"]:
    #     continue
    
    # metrics_per_corruption = get_metrics(v["df"].copy(), celebahq_fid)
    # display(Markdown(f"## {k}"))
    # display(metrics_per_corruption)

    # v["metrics_per_corruption"] = metrics_per_corruption

    _fake_df = v["df"].copy()
    _fake_df[_fake_df["corruption"].isin(["checkerboard_cutout", "perlin_noise", "masking_random_color", "masking_vline_random_color", "plasma_noise"])] 
    _fake_df["corruption"] = "all"

    metrics_aggregated = get_metrics(_fake_df.copy(), celebahq_fid)
    display(Markdown(f"## {k} - aggregated (only ablation)"))
    display(metrics_aggregated)
    
    v["metrics_aggregated"] = metrics_aggregated

  0%|          | 0/1 [00:00<?, ?it/s]

## reconstruction_without_clipping - aggregated (only ablation)

,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid,num_imgs
0,all,27.162848,14.31604,0.894763,0.475805,13.408826,78.688309,0.120334,0.589465,147.212692,1137


  0%|          | 0/1 [00:00<?, ?it/s]

## reconstruction_with_clipping - aggregated (only ablation)

...

## original - aggregated (only ablation)

,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid,num_imgs
0,all,14.746831,inf,0.563867,0.999998,73.49295,0.0,0.502755,0.0,48.817364,1206


  0%|          | 0/1 [00:00<?, ?it/s]

## corrupted - aggregated (only ablation)

,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid,num_imgs
0,all,inf,14.746831,1.0,0.563867,0.0,73.49295,0.0,0.502755,127.835823,1206


In [94]:
import pickle
with open("/home/tmartorella/imgs_for_metrics/metrics_new.pkl", "wb") as metrics_file:
    pickle.dump(dfs, metrics_file)

In [131]:
ode = df[(df["algo"] == "ode") & (df["corruption"].isin(["contrast", "fog"]))].drop(columns=["timestamp"])
ode_results_per_corruption = get_metrics(ode, celebahq_fid)
ode_results_per_corruption

100%|█████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:06<00:00, 33.48s/it]


,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid
0,contrast,19.456379,14.005204,0.704571,0.466285,37.227726,86.310837,0.566833,0.413938,51.584625
1,fog,18.370485,12.637938,0.599688,0.423423,49.261127,101.670357,0.444414,0.434942,50.019432


In [134]:
sde = df[(df["algo"] == "sde") & (df["corruption"].isin(["contrast", "fog"]))].drop(columns=["timestamp"])
sde_results_per_corruption = get_metrics(sde, celebahq_fid)
sde_results_per_corruption

100%|█████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:44<00:00, 22.06s/it]


,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid
0,contrast,31.507917,12.398882,0.825937,0.352600,9.956234,104.480896,0.535663,0.814965,330.311615
1,fog,26.394199,11.563744,0.707346,0.363064,20.327606,115.654778,0.416586,0.654471,196.135605


In [132]:
# ode = df[(df["algo"] == "ode") & (df["corruption"] == "fog")].drop(columns=["timestamp"])
ode_all = df[(df["algo"] == "ode")].copy()
ode_all["corruption"] = "all"
ode_results_all = get_metrics(ode_all, celebahq_fid)
ode_results_all

100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [04:12<00:00, 252.90s/it]


,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid
0,all,15.8325,14.758712,0.531907,0.51153,55.553547,69.620918,0.544291,0.349669,27.974178


In [133]:
sde_all = df[(df["algo"] == "sde")].copy()
sde_all["corruption"] = "all"
sde_results_all = get_metrics(sde_all, celebahq_fid)
sde_results_all

100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [05:42<00:00, 342.72s/it]


,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid
0,all,16.148903,14.417244,0.490403,0.517865,53.025021,72.267281,0.62709,0.368789,59.296925


In [128]:
# ode = df[(df["algo"] == "ode") & (df["corruption"] == "fog")].drop(columns=["timestamp"])
corrupted_all = df[(df["algo"] == "none") & (df["filename"] == "corrupted.png")].copy()
# corrupted_all["corruption"] = "all"
corrupted_results_all = get_metrics(corrupted_all, celebahq_fid)
corrupted_results_all

100%|█████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:01<00:00, 61.27s/it]


,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid
0,all,inf,12.579247,1.0,0.428789,0.0,94.061813,0.0,0.683682,113.806267


In [129]:
# ode = df[(df["algo"] == "ode") & (df["corruption"] == "fog")].drop(columns=["timestamp"])
corrupted_all = df[(df["algo"] == "none") & (df["filename"] == "corrupted.png")].copy()
# corrupted_all["corruption"] = "all"
corrupted_results_all = get_metrics(corrupted_all, celebahq_fid)
corrupted_results_all

100%|███████████████████████████████████████████████████████████████████████████████████████| 17/17 [01:41<00:00,  5.94s/it]


,corruption,psnr_to_source,psnr_to_target,ssim_to_source,ssim_to_target,l2_to_source,l2_to_target,lpips_to_source,lpips_to_target,fid
0,brightness,inf,9.852933,1.0,0.618916,0.0,140.778976,0.0,0.257829,98.784233
1,contrast,inf,12.546547,1.0,0.436881,0.0,102.690407,0.0,0.747116,112.282906
2,elastic_transform,inf,19.798203,1.0,0.566378,0.0,44.225132,0.0,0.254095,218.191284
3,fog,inf,11.679802,1.0,0.524399,0.0,114.024574,0.0,0.444118,183.971191
4,frost,inf,9.333415,1.0,0.436469,0.0,145.133148,0.0,0.561487,220.817673
5,gaussian_blur,inf,22.107393,1.0,0.618729,0.0,34.124580,0.0,0.609321,159.776321
6,gaussian_noise,inf,10.650517,1.0,0.058202,0.0,130.082214,0.0,1.331478,321.960663
7,glass_blur,inf,22.184576,1.0,0.622470,0.0,33.846889,0.0,0.461552,225.536407
8,impulse_noise,inf,10.400298,1.0,0.068457,0.0,133.768524,0.0,1.335539,288.324066
9,jpeg_compression,inf,25.767984,1.0,0.709763,0.0,22.577734,0.0,0.310088,125.927399


In [ ]:
ode_results_per_corruption.to_csv("/home/martorel/dev/unsupervised_image_editing/ode_results_per_corruption_top_16_all.csv")
sde_results_per_corruption.to_csv("/home/martorel/dev/unsupervised_image_editing/sde_results_per_corruption_top_16_all.csv")
ode_results_all.to_csv("/home/martorel/dev/unsupervised_image_editing/ode_results_all_top_16_all.csv")
sde_results_all.to_csv("/home/martorel/dev/unsupervised_image_editing/sde_results_all_top_16_all.csv")